# Importar bibliotecas a serem utilizadas

In [165]:
import pandas as pd
import datetime
from bs4 import BeautifulSoup
import requests
import re
import selenium
import locale
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# Inicializar selenium

In [166]:
options = Options()
#options.add_argument("--headless")
options.add_argument("--window-size=400x800")

## Inicializar chrome

In [ ]:
driver = webdriver.Chrome(options=options)

# Configurar Locale

In [167]:
locale.setlocale(locale.LC_ALL, 'pt_PT.UTF-8')

'pt_PT.UTF-8'

# Inicialização de variáveis

In [168]:
URL ='https://www.ncrangola.com/loja/particulares/pt/350-informatica#/page-{pagina}'


# Funções  auxiliares

In [6]:
def get_url(url):
    driver.get(url)
    #search_results = driver.find_element_by_xpath('/html/body/div[3]/div[1]/header/div[4]/div/div[1]/div/div[1]/a')
    #search_results.click()
    return driver.page_source

def click_link(url):
    source = get_url(url)
    site_data = BeautifulSoup(source, 'html.parser')
    return BeautifulSoup(site_data.prettify(), 'html.parser')


# def get_next_page():
   
#     number_of_pages = site.find('ul', attrs={'class':'pagination'}).find_all('li')
#     next_page = site.find('li', attrs={'class':'disabled pagination_next'})
#     i = 1
#     if not next_page and len(number_of_pages) > 1:
#             for page in range(1,len(number_of_pages)):
#                 #url = URL+ page.find('a')['href']
#                 if page < 6:
#                    return URL.format(pagina=page)
#                 else:
#                     return None
#     else:
#         return None



def get_produtos(page_source):
    produtos = []
    for produto in page_source:
        produtos.append({ 'Nome': produto.h5.get_text().strip(),
                        'Marca': produto.h6.get_text().strip(),
                        'Preco': produto.find('span', attrs={'itemprop':'price'}).text.strip(),
                        'Link': produto.find('a').get('href'),
                        'Imagem': produto.find('img').get('src'),
                        'dataConsulta': datetime.datetime.now()})
                        
                        
        # print("Nome",produto.h5.get_text(),sep=":")
        # print("Marca",produto.h6.get_text(),sep=":")
        # print("Preço",produto.find('span', attrs={'itemprop':'price'}).text,sep=":")
        # print("Link",produto.find('a').get('href'),sep=":")
        # print("Imagem",produto.find('img').get('src'),sep=":")
    
    return produtos

In [ ]:
# while True:
#     url = get_next_page()
#     if url:
#         data = click_link(url)
#         print(data)
#     else:
#         break

## Tratamento dos dados

In [149]:
df_produtos = pd.DataFrame(columns=['Nome','Marca','Preco','Link','Imagem','dataConsulta'])

In [150]:
   
for page in range(1,6):
    url = URL.format(pagina=page)
    data = click_link(url)
    produtos = data.find_all('div', attrs={'class':'produto_geral'})
    dataset= get_produtos(produtos)
    #print(dataset)
    df_produtos = df_produtos.append(dataset,ignore_index=True)


In [151]:
df_produtos.count()

Nome            300
Marca           300
Preco           300
Link            300
Imagem          300
dataConsulta    300
dtype: int64

In [152]:
df_produtos.replace(' ', '', regex=True, inplace=True)

In [153]:
df_produtos.replace('AKZ', ' €', regex=True, inplace=True)

In [154]:
df_produtos['Preco'] = df_produtos['Preco'].apply(lambda x: locale.atof(x.split()[0]))

In [155]:
df_produtos.dtypes

Nome                    object
Marca                   object
Preco                  float64
Link                    object
Imagem                  object
dataConsulta    datetime64[ns]
dtype: object

In [156]:
df_produtos.head()

,Nome,Marca,Preco,Link,Imagem,dataConsulta
0,LEITORDECARTÃOENTRA-SAIDASD/MICROSD/XDP/CXATX,NCR,7115.0,https://www.ncrangola.com/loja/particulares/pt/cartoes-de-memoria/13757-leitor-de-cartes-sdmicrosdxd-pcx-atx.html,https://www.ncrangola.com/loja/particulares/img/p/pt-default-home_default.jpg,2021-11-19 12:34:00.713561
1,BOLSA92PARACDs,CASELOGIC,7137.0,https://www.ncrangola.com/loja/particulares/pt/bolsas/12218-bolsa-case-logic-92-cd-preto.html,https://www.ncrangola.com/loja/particulares/13219-home_default/bolsa-case-logic-92-cd-preto.jpg,2021-11-19 12:34:00.713736
2,CABOUSBPARAUSB-CJELLEEZORANGE,MAXELL,7143.0,https://www.ncrangola.com/loja/particulares/pt/cabos-/17695-cabo-maxell-usb-para-usb-c-jelleez-orange-.html,https://www.ncrangola.com/loja/particulares/27908-home_default/cabo-maxell-usb-para-usb-c-jelleez-orange-.jpg,2021-11-19 12:34:00.713906
3,CABO4FTUSB-APARAUSB-CJELLEEZPRETO348215,MAXELL,7144.0,https://www.ncrangola.com/loja/particulares/pt/cabos-/16718-cabo-maxell-4ft-usb-to-usbc-jelleez-blk-348215.html,https://www.ncrangola.com/loja/particulares/25447-home_default/cabo-maxell-4ft-usb-to-usbc-jelleez-blk-348215.jpg,2021-11-19 12:34:00.714074
4,CABOUSB-APARAMICRO-USBTEXTILEDOURADO,CELLY,7169.0,https://www.ncrangola.com/loja/particulares/pt/cabos-/17831-cabo-celly-usb-to-micro-usb-textile-gold.html,https://www.ncrangola.com/loja/particulares/28153-home_default/cabo-celly-usb-to-micro-usb-textile-gold.jpg,2021-11-19 12:34:00.714241


## Formatação de dados pandas

In [158]:
formato = { 'Preco': '{0:,.2f} Kz'}
df_produtos.head().style.format(formato)

,Nome,Marca,Preco,Link,Imagem,dataConsulta
0,LEITORDECARTÃOENTRA-SAIDASD/MICROSD/XDP/CXATX,NCR,"7,115.00 Kz",https://www.ncrangola.com/loja/particulares/pt/cartoes-de-memoria/13757-leitor-de-cartes-sdmicrosdxd-pcx-atx.html,https://www.ncrangola.com/loja/particulares/img/p/pt-default-home_default.jpg,2021-11-19 12:34:00.713561
1,BOLSA92PARACDs,CASELOGIC,"7,137.00 Kz",https://www.ncrangola.com/loja/particulares/pt/bolsas/12218-bolsa-case-logic-92-cd-preto.html,https://www.ncrangola.com/loja/particulares/13219-home_default/bolsa-case-logic-92-cd-preto.jpg,2021-11-19 12:34:00.713736
2,CABOUSBPARAUSB-CJELLEEZORANGE,MAXELL,"7,143.00 Kz",https://www.ncrangola.com/loja/particulares/pt/cabos-/17695-cabo-maxell-usb-para-usb-c-jelleez-orange-.html,https://www.ncrangola.com/loja/particulares/27908-home_default/cabo-maxell-usb-para-usb-c-jelleez-orange-.jpg,2021-11-19 12:34:00.713906
3,CABO4FTUSB-APARAUSB-CJELLEEZPRETO348215,MAXELL,"7,144.00 Kz",https://www.ncrangola.com/loja/particulares/pt/cabos-/16718-cabo-maxell-4ft-usb-to-usbc-jelleez-blk-348215.html,https://www.ncrangola.com/loja/particulares/25447-home_default/cabo-maxell-4ft-usb-to-usbc-jelleez-blk-348215.jpg,2021-11-19 12:34:00.714074
4,CABOUSB-APARAMICRO-USBTEXTILEDOURADO,CELLY,"7,169.00 Kz",https://www.ncrangola.com/loja/particulares/pt/cabos-/17831-cabo-celly-usb-to-micro-usb-textile-gold.html,https://www.ncrangola.com/loja/particulares/28153-home_default/cabo-celly-usb-to-micro-usb-textile-gold.jpg,2021-11-19 12:34:00.714241


## Análise de dados

In [159]:
df_produtos.groupby('Marca').count()

,Nome,Preco,Link,Imagem,dataConsulta
Marca,,,,,
AVERY,5,5,5,5,5
CASELOGIC,5,5,5,5,5
CELLY,30,30,30,30,30
DLINK,10,10,10,10,10
FELLOWES,10,10,10,10,10
GENIUS,35,35,35,35,35
HP,10,10,10,10,10
INTELLINET,15,15,15,15,15
KINGSLONG,30,30,30,30,30


In [160]:
len(df_produtos)

300

In [161]:
df_produtos.groupby('Marca')['Preco'].agg(['min','max','mean','median','std']).style.format('{0:,.2f} Kz')

,min,max,mean,median,std
Marca,,,,,
AVERY,"9,827.00 Kz","9,827.00 Kz","9,827.00 Kz","9,827.00 Kz",0.00 Kz
CASELOGIC,"7,137.00 Kz","7,137.00 Kz","7,137.00 Kz","7,137.00 Kz",0.00 Kz
CELLY,"7,169.00 Kz","9,804.00 Kz","8,372.50 Kz","8,144.50 Kz","1,245.85 Kz"
DLINK,"8,796.24 Kz","9,951.00 Kz","9,373.62 Kz","9,373.62 Kz",608.61 Kz
FELLOWES,"8,707.32 Kz","9,462.00 Kz","9,084.66 Kz","9,084.66 Kz",397.75 Kz
GENIUS,"7,249.00 Kz","8,705.00 Kz","8,137.87 Kz","8,705.00 Kz",676.01 Kz
HP,"8,167.00 Kz","8,306.00 Kz","8,236.50 Kz","8,236.50 Kz",73.26 Kz
INTELLINET,"8,268.00 Kz","9,528.00 Kz","9,093.67 Kz","9,485.00 Kz",604.60 Kz
KINGSLONG,"8,384.00 Kz","8,934.00 Kz","8,590.33 Kz","8,453.00 Kz",252.10 Kz


## Exportação dos dados para excel

In [162]:
df_produtos.to_excel('NCR_produtos_1.xlsx',index=False)